## VQE PARA EL CÁLCULO DEL PERFIL DE DISOCIACIÓN DE UNA MOLÉCULA DE $H_2$

Vamos a usar el Variational Quantum Eigensolver para calcular el perfil de disociación de una molécula de $H_2$. Comenzamos importando los paquetes necesarios

In [ ]:
from qiskit import Aer
from qiskit_nature.algorithms import GroundStateEigensolver
from qiskit_nature.drivers import UnitsType, Molecule
from qiskit_nature.drivers.second_quantization import ElectronicStructureDriverType, ElectronicStructureMoleculeDriver
from qiskit_nature.problems.second_quantization import ElectronicStructureProblem
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.mappers.second_quantization import JordanWignerMapper
from qiskit.algorithms.minimum_eigen_solvers import NumPyMinimumEigensolver, VQE
from qiskit.providers.aer import StatevectorSimulator
from qiskit.utils import QuantumInstance
from qiskit_nature.algorithms import VQEUCCFactory
import numpy as np
import matplotlib.pyplot as plt

## Obteniendo el hamiltoniano y encontrando su estado de mínima energía

Definimos la molécula y los conversores necesarios para pasar de hamiltoniano fermiónico a hamiltoniano de qubits

In [ ]:
molecule = Molecule(geometry=[['H', [0., 0., 0.]],
                              ['H', [0., 0., 0.735]]],
                     charge=0, multiplicity=1)

driver = ElectronicStructureMoleculeDriver(molecule, basis='sto3g', driver_type=ElectronicStructureDriverType.PYSCF)
es_problem = ElectronicStructureProblem(driver)
qubit_converter = QubitConverter(JordanWignerMapper())
second_q_op = es_problem.second_q_ops()
print(second_q_op[0])
qubit_op = qubit_converter.convert(second_q_op[0])
print("Hamiltoniano de qubits")
print(qubit_op)

Ahora usamos VQE para encontrar el estado de mínima energía. La clase *VQEUCCFactory* usa el ansatz *UCCSD* (Unitary Coupled-Cluster Singles and Doubles)

In [ ]:
quantum_instance = QuantumInstance(backend = Aer.get_backend('aer_simulator_statevector'))
vqe_solver = VQEUCCFactory(quantum_instance)
calc = GroundStateEigensolver(qubit_converter, vqe_solver)
res = calc.solve(es_problem)
print(res)

## Calculando el perfil de disociación

Ahora vamos a ir variando la distancia entre los átomos y calculando la energía en función de esa distancia. Compararemos los resultados de VQE con los de la solución exacta

In [ ]:
numpy_solver = NumPyMinimumEigensolver() # Para obtener la solución exacta

distances = np.linspace(0.25, 3.0, 30)
exact_energies = []
vqe_energies = []
for dist in distances:
    molecule = Molecule(geometry=[['H', [0., 0., 0.]],
                              ['H', [0., 0., dist]]],
                     charge=0, multiplicity=1)
    driver = ElectronicStructureMoleculeDriver(molecule, basis='sto3g', driver_type=ElectronicStructureDriverType.PYSCF)
    es_problem = ElectronicStructureProblem(driver)
    # Exact solver
    calc = GroundStateEigensolver(qubit_converter, numpy_solver)
    res = calc.solve(es_problem)
    exact_energies.append(res.total_energies)
    # VQE
    calc = GroundStateEigensolver(qubit_converter, vqe_solver)
    res = calc.solve(es_problem)
    vqe_energies.append(res.total_energies)

Representamos gráficamente los resultados

In [ ]:
plt.plot(exact_energies, label = 'Exact solver')
plt.plot(vqe_energies, label = 'VQE')
plt.title('Perfil de disociación')
plt.xlabel('Distancia interatómica')
plt.legend()
plt.ylabel('Energía');